---
title: Parker Solar Probe (PSP)
---

In [ ]:
# | default_exp missions/psp
# | export
from typing import Literal
from space_analysis.core import Instrument, Mission

In [ ]:
# | export
fields = Instrument(name="FIELDS")
spc = Instrument(name="SPC")
spi = Instrument(name="SPI")

psp = Mission(
    name="Parker Solar Probe", instruments=[fields, spc, spi], names_list=["psp"]
)

MagDataVars = Literal["psp_fld_l2_mag_RTN",]

SPIDataVars = Literal[
    "QUALITY_FLAG",
    "DENS",
    "VEL_INST",
    "VEL_SC",
    "VEL_RTN_SUN",
    "T_TENSOR_INST",
    "TEMP",  # Temperature (eV)
    "EFLUX_VS_ENERGY",
    "EFLUX_VS_THETA",
    "EFLUX_VS_PHI",
    "SUN_DIST",
    "VENUS_DIST",
    "SC_VEL_RTN_SUN",
    "QUAT_SC_TO_RTN",
    "MAGF_SC",  # Magnetic Field in Spacecraft Coordinates
    "MAGF_INST",
]

In [ ]:
# | export
def validate(instrument, datatype=None, var_names=None, **kwargs):
    "validate the input arguments and return the correct ones"
    if instrument == "fields":
        datatype = datatype or "mag_rtn"
        var_names = var_names or ["psp_fld_l2_mag_RTN"]
    elif instrument == "spi":
        datatype = datatype or "sf00_l3_mom"
        var_names = var_names or ["DENS", "VEL_RTN_SUN", "TEMP"]
    return var_names, datatype

In [ ]:
import polars as pl
import pyspedas

from functools import partial
from space_analysis.io.cdf import cdf2pl
from typing import overload

In [ ]:
def download_data(**kwargs):
    return pyspedas.psp.load(downloadonly=True, **kwargs)


@overload
def load(
    instrument: Literal["fields"],
    var_names: list[MagDataVars],
    datatype: Literal["mag_rtn"],
    level: Literal["l2"],
    **kwargs,
): ...


@overload
def load(
    instrument: Literal["spi"],
    var_names: list[SPIDataVars],
    datatype: Literal["sf00_l3_mom"],
    level: Literal["l3"],
    **kwargs,
): ...


def load(instrument="fields", var_names=None, datatype=None, **kwargs) -> pl.LazyFrame:
    var_names, datatype = validate(instrument, datatype, var_names, **kwargs)

    files = download_data(instrument=instrument, datatype=datatype, **kwargs)

    load_func = partial(cdf2pl, var_names=var_names)

    return pl.concat(load_func(file) for file in files)